In [1]:
import os, sys
import numpy as np

import fitsio
import matplotlib.pyplot as plt

from astropy.table import Table, Column

from desisim.io import read_simspec
from desisim.scripts import quickgen
import desispec.io
from desispec import brick   

from desispec.log import get_logger
log = get_logger()

%matplotlib inline

In [2]:
def _default_wave(wavemin=None, wavemax=None, dw=0.2):
    '''Construct and return the default wavelength vector.'''
    import desimodel.io
    if wavemin is None: 
        wavemin = desimodel.io.load_throughput('b').wavemin
    if wavemax is None:
        wavemax = desimodel.io.load_throughput('z').wavemax
    wave = np.arange(round(wavemin, 1), wavemax, dw)
                                                                                                              
    return wave

In [3]:
def get_lensing_targets(priors, tileid=None, seed=None, specmin=0):
    """
    Returns:
        fibermap
        truth table
    TODO: document this better
    
    """
    import desimodel.io
    from desimodel.focalplane import FocalPlane
    from desispec.io.fibermap import empty_fibermap
    from desisim.io import empty_metatable, get_tile_radec
    from desisim.templates import ELG, LRG

    if tileid is None:
        tile_ra, tile_dec = 0.0, 0.0
    else:
        tile_ra, tile_dec = get_tile_radec(tileid)  
    
    log.debug('Using random seed {}'.format(seed))
    rand = np.random.RandomState(seed)
    
    # Get DESI wavelength coverage.
    wave = _default_wave()
    nwave = len(wave)
    
    # Initialize the 'truth' dictionary and output metadata and fibermap tables.
    nspec = priors['NSPEC']
    truth = dict()
    truth['FLUX'] = np.zeros((nspec, len(wave)))
    truth['OBJTYPE'] = np.zeros(nspec, dtype=(str, 10))
    truth['WAVE'] = wave

    objtype = 'LRG'
    truth['META'] = empty_metatable(nmodel=nspec, objtype=objtype)
    truth['OBJTYPE'] = objtype

    fibermap = empty_fibermap(nspec)
    fibermap['OBJTYPE'] = objtype
    
    # Simulate spectra according to the input priors.
    elg = ELG(wave=wave)
    lrg = LRG(wave=wave)

    esimflux, _, emeta = elg.make_templates(nmodel=nspec, seed=seed, 
                                            zrange=priors['ZRANGE_SOURCE'],
                                            rmagrange=priors['MAGRANGE_SOURCE'],
                                            nocolorcuts=True, novdisp=True)
    lsimflux, _, lmeta = lrg.make_templates(nmodel=nspec, seed=seed,
                                            zrange=priors['ZRANGE_LENS'],
                                            zmagrange=priors['MAGRANGE_LENS'],
                                            nocolorcuts=True, novdisp=True)
    simflux = lsimflux + esimflux
    log.debug('This is wrong! Using the LRG table for META.')
    meta = lmeta 

    truth['FLUX'] = 1e17 * simflux
    truth['UNITS'] = '1e-17 erg/s/cm2/A'
    truth['META'] = meta
    truth['META_SOURCE'] = emeta

    ugrizy = 22.5-2.5*np.log10(meta['DECAM_FLUX'].data)
    wise = 22.5-2.5*np.log10(meta['WISE_FLUX'].data)
    fibermap['FILTER'][:, :6] = ['DECAM_G', 'DECAM_R', 'DECAM_Z', 'WISE_W1', 'WISE_W2']
    fibermap['MAG'][:, 0] = ugrizy[:, 1]
    fibermap['MAG'][:, 1] = ugrizy[:, 2]
    fibermap['MAG'][:, 2] = ugrizy[:, 4]
    fibermap['MAG'][:, 3] = wise[:, 0]
    fibermap['MAG'][:, 4] = wise[:, 1]

    # Load fiber -> positioner mapping and tile information
    fiberpos = desimodel.io.load_fiberpos()

    # Where are these targets?  Centered on positioners for now.
    x = fiberpos['X'][specmin:specmin+nspec]
    y = fiberpos['Y'][specmin:specmin+nspec]
    fp = FocalPlane(tile_ra, tile_dec)
    ra = np.zeros(nspec)
    dec = np.zeros(nspec)
    for i in range(nspec):
        ra[i], dec[i] = fp.xy2radec(x[i], y[i])

    # Fill in the rest of the fibermap structure
    fibermap['FIBER'] = np.arange(nspec, dtype='i4')
    fibermap['POSITIONER'] = fiberpos['POSITIONER'][specmin:specmin+nspec]
    fibermap['SPECTROID'] = fiberpos['SPECTROGRAPH'][specmin:specmin+nspec]
    fibermap['TARGETID'] = rand.randint(sys.maxsize, size=nspec)
    fibermap['TARGETCAT'] = np.zeros(nspec, dtype=(str, 20))
    fibermap['LAMBDAREF'] = np.ones(nspec, dtype=np.float32)*5400
    fibermap['RA_TARGET'] = ra
    fibermap['DEC_TARGET'] = dec
    fibermap['X_TARGET'] = x
    fibermap['Y_TARGET'] = y
    fibermap['X_FVCOBS'] = fibermap['X_TARGET']
    fibermap['Y_FVCOBS'] = fibermap['Y_TARGET']
    fibermap['X_FVCERR'] = np.zeros(nspec, dtype=np.float32)
    fibermap['Y_FVCERR'] = np.zeros(nspec, dtype=np.float32)
    fibermap['RA_OBS'] = fibermap['RA_TARGET']
    fibermap['DEC_OBS'] = fibermap['DEC_TARGET']
    fibermap['BRICKNAME'] = brick.brickname(ra, dec)

    return fibermap, truth

In [4]:
def new_lensing_exposure(priors, airmass=1.0, exptime=None, night=None, tileid=None,
                         expid=None, seed=None, outdir='./'):
    """
    Create a new exposure and output input simulation files.
    Does not generate pixel-level simulations or noisy spectra.

    Args:
        priors (astropy.Table): table of prior parameters

    Options:
        airmass : airmass, default 1.0
        exptime : exposure time in seconds
        seed : random seed
        outdir : output directory

    Writes:
        $OUTDIR/fibermap-{expid}.fits
        $OUTDIR/simspec-{expid}.fits

    Returns:
        fibermap numpy structured array
        truth dictionary

    Notes:

    """
    import time
    import desimodel.io
    from desisim.io import get_tile_radec, write_simspec
    from desisim.obs import get_next_expid, get_next_tileid, get_night

    if expid is None:
        expid = get_next_expid()
    if tileid is None:
        tileid = get_next_tileid()
    if night is None:
        night = get_night(utc=time.gmtime()) # simulation obs time = now, even if sun is up
    dateobs = time.strptime(night+':22', '%Y%m%d:%H')
    
    flavor = 'science'
    
    params = desimodel.io.load_desiparams()
    if exptime is None:
        exptime = params['exptime']
        
    nspec = priors['NSPEC']
    log.debug('Generating {} targets'.format(nspec))
    fibermap, truth = get_lensing_targets(priors, tileid=tileid, seed=seed)

    flux = truth['FLUX']
    wave = truth['WAVE']
    nwave = len(wave)

    # Load sky [Magic knowledge of units 1e-17 erg/s/cm2/A/arcsec2]
    skyfile = os.getenv('DESIMODEL')+'/data/spectra/spec-sky.dat'

    log.info('skyfile {}'.format(skyfile))
    skywave, skyflux = np.loadtxt(skyfile, unpack=True)
    skyflux = np.interp(wave, skywave, skyflux)
    truth['SKYFLUX'] = skyflux

    log.debug('Calculating flux -> photons')
    for channel in ('B', 'R', 'Z'):
        thru = desimodel.io.load_throughput(channel)

        ii = np.where( (thru.wavemin <= wave) & (wave <= thru.wavemax) )[0]

        # Project flux to photons
        phot = thru.photons(wave[ii], flux[:,ii], units=truth['UNITS'],
                objtype='LRG', exptime=exptime, airmass=airmass)

        truth['PHOT_'+channel] = phot
        truth['WAVE_'+channel] = wave[ii]

        # Project sky flux to photons
        skyphot = thru.photons(wave[ii], skyflux[ii]*airmass,
                               units='1e-17 erg/s/cm2/A/arcsec2',
                               objtype='SKY', exptime=exptime, airmass=airmass)

        # 2D version
        ### truth['SKYPHOT_'+channel] = np.tile(skyphot, nspec).reshape((nspec, len(ii)))
        # 1D version
        truth['SKYPHOT_{}'.format(channel)] = skyphot.astype(np.float32)

    # NOTE: someday skyflux and skyphot may be 2D instead of 1D

    # Extract the metadata part of the truth dictionary into a table
    meta = truth['META']

    # Write fibermap
    telera, teledec = get_tile_radec(tileid)
    hdr = dict(
        NIGHT = (night, 'Night of observation YEARMMDD'),
        EXPID = (expid, 'DESI exposure ID'),
        TILEID = (tileid, 'DESI tile ID'),
        FLAVOR = (flavor, 'Flavor [arc, flat, science, ...]'),
        TELRA = (telera, 'Telescope pointing RA [degrees]'),
        TELDEC = (teledec, 'Telescope pointing dec [degrees]'),
        )
    #- ISO 8601 DATE-OBS year-mm-ddThh:mm:ss
    fiberfile = desispec.io.findfile('fibermap', night, expid)#, specprod_dir=outdir)
    print(fiberfile)
    #fiberfile = os.path.join(outdir, 'fibermap-{:08d}.fits'.format(expid))
    desispec.io.write_fibermap(fiberfile, fibermap, header=hdr)
    log.info('Wrote {}'.format(fiberfile))

    #- Write simspec; expand fibermap header
    hdr['AIRMASS'] = (airmass, 'Airmass at middle of exposure')
    hdr['EXPTIME'] = (exptime, 'Exposure time [sec]')
    hdr['DATE-OBS'] = (time.strftime('%FT%T', dateobs), 'Start of exposure')

    #simfile = os.path.join(outdir, night, 'simspec-{:08d}.fits'.format(expid))
    simfile = write_simspec(meta, truth, expid, night, header=hdr)#, outfile=simfile)
    log.info('Wrote {}'.format(simfile))

    return fibermap, truth

In [5]:
# Initialize the table of prior parameters.
lensdir = os.path.join(os.getenv('HOME'), 'simlens')
print(lensdir)

# Temporarily hack the DESI environments
os.environ['DESI_SPECTRO_SIM'] = lensdir
os.environ['DESI_SPECTRO_DATA'] = lensdir
os.environ['DESI_SPECTRO_REDUX'] = lensdir
os.environ['PIXPROD'] = ''
os.environ['SPECPROD'] = ''

#%env DESI_SPECTRO_DATA=lensdir
#%env DESI_SPECTRO_REDUX=lensdir
#%env DESI_SPECTRO_SIM=lensdir
#%env PIXPROD=
#%env SPECPROD=

seed = 456
rand = np.random.RandomState(seed)

# Make our table.
night = '20170108'
tileid = 1000
nspec = 3
exptime = np.array([10000.0, 100000.0]).astype('f4')
nexp = len(exptime)
expid = np.arange(nexp)

priors = dict()
priors['LENSDIR'] = lensdir
priors['NIGHT'] = night
priors['TILEID'] = tileid
priors['EXPID'] = expid
priors['EXPTIME'] = exptime
priors['SEED'] = seed
priors['NSPEC'] = nspec
priors['ZRANGE_SOURCE'] = np.array([1.0, 1.0])
priors['ZRANGE_LENS'] = np.array([0.5, 0.5])
priors['MAGRANGE_SOURCE'] = np.array([21.0, 21.0])
priors['MAGRANGE_LENS'] = np.array([21.3, 21.3])
print(priors)

/home/desi1/simlens
{'EXPTIME': array([  10000.,  100000.], dtype=float32), 'MAGRANGE_SOURCE': array([ 21.,  21.]), 'MAGRANGE_LENS': array([ 21.3,  21.3]), 'EXPID': array([0, 1]), 'NIGHT': '20170108', 'LENSDIR': '/home/desi1/simlens', 'TILEID': 1000, 'NSPEC': 3, 'ZRANGE_SOURCE': array([ 1.,  1.]), 'ZRANGE_LENS': array([ 0.5,  0.5]), 'SEED': 456}


In [6]:
# Generate fibermap and simspec files.
lensdir = priors['LENSDIR']
for ii in range(nexp):
    fibermap, truth = new_lensing_exposure(priors, exptime=priors['EXPTIME'][ii], night=priors['NIGHT'],
                                           tileid=priors['TILEID'], expid=priors['EXPID'][ii], seed=seed, 
                                           outdir=lensdir)
    # Write out truth table of the source.
    #source_meta = truth['META_SOURCE']


INFO:io.py:622:read_basis_templates: Reading /global/work/desi/spectro/templates/basis_templates/v2.2/elg_templates_v2.0.fits
INFO:io.py:622:read_basis_templates: Reading /global/work/desi/spectro/templates/basis_templates/v2.2/lrg_templates_v1.3.fits
INFO:<ipython-input-4-6910855acf4f>:57:new_lensing_exposure: skyfile /usr/local/desihub/desimodel/data/spectra/spec-sky.dat
/home/desi1/simlens/20170108/fibermap-00000000.fits
INFO:<ipython-input-4-6910855acf4f>:105:new_lensing_exposure: Wrote /home/desi1/simlens/20170108/fibermap-00000000.fits
INFO:<ipython-input-4-6910855acf4f>:114:new_lensing_exposure: Wrote /home/desi1/simlens/20170108/simspec-00000000.fits


INFO:io.py:622:read_basis_templates: Reading /global/work/desi/spectro/templates/basis_templates/v2.2/elg_templates_v2.0.fits
INFO:io.py:622:read_basis_templates: Reading /global/work/desi/spectro/templates/basis_templates/v2.2/lrg_templates_v1.3.fits
INFO:<ipython-input-4-6910855acf4f>:57:new_lensing_exposure: skyfile /usr/local/desihub/desimodel/data/spectra/spec-sky.dat
/home/desi1/simlens/20170108/fibermap-00000001.fits
INFO:<ipython-input-4-6910855acf4f>:105:new_lensing_exposure: Wrote /home/desi1/simlens/20170108/fibermap-00000001.fits
INFO:<ipython-input-4-6910855acf4f>:114:new_lensing_exposure: Wrote /home/desi1/simlens/20170108/simspec-00000001.fits


In [7]:
os.getenv('PIXPROD')
os.getenv('DESI_SPECTRO_DATA')

'/home/desi1/simlens'

In [8]:
# Run quickgen
lensdir = priors['LENSDIR']
for ii in range(nexp):
    expid = priors['EXPID'][ii]
    #fiberfile = 'fibermap-{:08d}.fits'.format(expid)
    #simfile = 'simspec-{:08d}.fits'.format(expid)
    fiberfile = os.path.join(lensdir, night, 'fibermap-{:08d}.fits'.format(expid))
    simfile = os.path.join(lensdir, night, 'simspec-{:08d}.fits'.format(expid))
    print(fiberfile, simfile)
    quickargs = quickgen.parse(['--simspec', simfile, '--fibermap', fiberfile])
    quickgen.main(quickargs)


/home/desi1/simlens/20170108/fibermap-00000000.fits /home/desi1/simlens/20170108/simspec-00000000.fits
INFO:quickgen.py:242:main: Reading fibermap file /home/desi1/simlens/20170108/fibermap-00000000.fits
INFO:quickgen.py:275:main: Initializing SpecSim with config desi
INFO:quickgen.py:281:main: Reading input file /home/desi1/simlens/20170108/simspec-00000000.fits
INFO:quickgen.py:310:main: Only 3 spectra in input file
INFO:quickgen.py:670:main: Writing files for channel:b, spectrograph:0, spectra:0 to 3
INFO:quickgen.py:696:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/frame-b0-00000000.fits
INFO:quickgen.py:713:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/cframe-b0-00000000.fits
INFO:quickgen.py:728:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/sky-b0-00000000.fits
INFO:quickgen.py:749:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/calib-b0-00000000.fits
INFO:quickgen.py:670:main: Writing files for channel:r, 

INFO:quickgen.py:696:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/frame-r0-00000000.fits
INFO:quickgen.py:713:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/cframe-r0-00000000.fits
INFO:quickgen.py:728:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/sky-r0-00000000.fits
INFO:quickgen.py:749:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/calib-r0-00000000.fits
INFO:quickgen.py:670:main: Writing files for channel:z, spectrograph:0, spectra:0 to 3
INFO:quickgen.py:696:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/frame-z0-00000000.fits
INFO:quickgen.py:713:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/cframe-z0-00000000.fits
INFO:quickgen.py:728:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/sky-z0-00000000.fits
INFO:quickgen.py:749:main: Wrote file /home/desi1/simlens/exposures/20170108/00000000/calib-z0-00000000.fits
/home/desi1/simlens/20170108/fibermap-00000

INFO:quickgen.py:713:main: Wrote file /home/desi1/simlens/exposures/20170108/00000001/cframe-r0-00000001.fits
INFO:quickgen.py:728:main: Wrote file /home/desi1/simlens/exposures/20170108/00000001/sky-r0-00000001.fits
INFO:quickgen.py:749:main: Wrote file /home/desi1/simlens/exposures/20170108/00000001/calib-r0-00000001.fits
INFO:quickgen.py:670:main: Writing files for channel:z, spectrograph:0, spectra:0 to 3
INFO:quickgen.py:696:main: Wrote file /home/desi1/simlens/exposures/20170108/00000001/frame-z0-00000001.fits
INFO:quickgen.py:713:main: Wrote file /home/desi1/simlens/exposures/20170108/00000001/cframe-z0-00000001.fits
INFO:quickgen.py:728:main: Wrote file /home/desi1/simlens/exposures/20170108/00000001/sky-z0-00000001.fits
INFO:quickgen.py:749:main: Wrote file /home/desi1/simlens/exposures/20170108/00000001/calib-z0-00000001.fits


In [14]:
# Make bricks from cframe files.
from desispec.scripts import makebricks

makeargs = makebricks.parse(['--night', priors['NIGHT'], '--verbose'])
makebricks.main(makeargs)



DEBUG:makebricks.py:59:main: Exposure 00000000 covers 3 bricks and has cframes for b0,z0,r0.
DEBUG:makebricks.py:84:main: Brick /home/desi1/simlens/bricks/3587m005/brick-b-3587m005.fits now contains 3 spectra for 1 targets.
DEBUG:makebricks.py:84:main: Brick /home/desi1/simlens/bricks/3592m005/brick-b-3592m005.fits now contains 3 spectra for 1 targets.
DEBUG:makebricks.py:84:main: Brick /home/desi1/simlens/bricks/3587p000/brick-b-3587p000.fits now contains 3 spectra for 1 targets.
DEBUG:makebricks.py:84:main: Brick /home/desi1/simlens/bricks/3587m005/brick-z-3587m005.fits now contains 3 spectra for 1 targets.
DEBUG:makebricks.py:84:main: Brick /home/desi1/simlens/bricks/3592m005/brick-z-3592m005.fits now contains 3 spectra for 1 targets.
DEBUG:makebricks.py:84:main: Brick /home/desi1/simlens/bricks/3587p000/brick-z-3587p000.fits now contains 3 spectra for 1 targets.
DEBUG:makebricks.py:84:main: Brick /home/desi1/simlens/bricks/3587m005/brick-r-3587m005.fits now contains 3 spectra for 1

In [40]:
# Figure out redshift fitting.
from desispec.scripts import zfind

# Need code to get bricks and bricknames automatically.
bricks = [('3587m005')]

nproc = 4
zrange = [0.45, 0.55]

for bb in list(bricks):
    print(bb)
    brickfiles = [os.path.join(lensdir, 'bricks', bb, 'brick-{}-{}.fits'.format(ch, bb)) for ch in ['b', 'r', 'z']]
    #print(brickfiles)
    zargs = zfind.parse([
        '--brick', bb, 
        '--nproc', '{}'.format(nproc),
        '--zrange-galaxy', '{}'.format(zrange[0]), '{}'.format(zrange[1]), 
        #'--zspec', 
        #'--qafile', os.path.join(lensdir, 'bricks', bb, 'qa-zbest-{}.pdf'.format(bb)),
        #'--outfile', os.path.join(lensdir, 'zbest-{}.fits'.format(exp))
        '--objtype', 'LRG'])
    zfind.main(zargs)


3587m005
INFO:zfind.py:89:main: Reading bricks
INFO:zfind.py:115:main: Filter found: b
INFO:zfind.py:115:main: Filter found: r
INFO:zfind.py:115:main: Filter found: z
INFO:zfind.py:129:main: Combining individual channels and exposures
INFO:zfind.py:197:main: number of good targets = 1
INFO:zfind.py:207:main: Fitting 1 targets
INFO fitted fiber 1/1, chi2_null=1760440.484892, 300 templates in /usr/local/desihub/redmonster/templates/ndArch-ssp_em_galaxy-v000.fits, npoly=2, using 4 procs in 2.565215 sec
INFO Not writing chi2
INFO:zfind.py:372:main: Writing /home/desi1/simlens/bricks/3587m005/zbest-3587m005.fits


In [39]:
thisbrick = '3587m005'
zfile = os.path.join(lensdir, 'bricks', thisbrick, 'zbest-{}.fits'.format(thisbrick))
zbest = Table(fitsio.read(zfile, ext=1))
print(zbest)

simfile = os.path.join(lensdir, priors['NIGHT'], 'simspec-{:08d}.fits'.format(priors['EXPID'][0]))
sim1 = read_simspec(simfile)
print(Table(sim1.metadata))



BRICKNAME       TARGETID            Z       ... ZWARN    SPECTYPE   SUBTYPE
--------- ------------------- ------------- ... ----- ------------- -------
 3587m005 5232294543435377135 0.53345210212 ...     0 ssp_em_galaxy      NA
OBJTYPE SUBTYPE TEMPLATEID    SEED    REDSHIFT ... ZMETAL   AGE   TEFF LOGG FEH 
------- ------- ---------- ---------- -------- ... ------ ------- ---- ---- ----
    LRG                 78 1068398491      0.5 ...  0.019 5.62341 -1.0 -1.0 -1.0
    LRG                135 4211147365      0.5 ...   0.03 9.44061 -1.0 -1.0 -1.0
    LRG                 67  700366507      0.5 ...  0.019 2.98538 -1.0 -1.0 -1.0


In [ ]:
# Below here requires hacking....


In [9]:
# Read the simspec file to get trueflux.
simfile = os.path.join(lensdir, 'simspec-{:08d}.fits'.format(0))
sim = read_simspec(simfile)

simfile1 = os.path.join(lensdir, 'simspec-{:08d}.fits'.format(1))
sim1 = read_simspec(simfile1)

FileNotFoundError: [Errno 2] No such file or directory: '/home/desi1/simlens/simspec-00000000.fits'

In [ ]:
for channel in ('b', 'r', 'z'):
    ff = desispec.io.read_frame(os.path.join(lensdir, 'cframe-{}0-{:08d}.fits'.format(channel, 0)))
    plt.plot(ff.wave, ff.flux[0, :])
    
    ff_test = desispec.io.read_frame(os.path.join(lensdir, 'cframe-{}0-{:08d}.fits'.format(channel, 1)))
    plt.plot(ff_test.wave, ff_test.flux[0, :])

plt.plot(sim.wave['brz'], sim.flux[0, :], color='k')
plt.ylim(-4, 10)    
plt.show()

for channel in ('b', 'r', 'z'):
    ff = desispec.io.read_frame(os.path.join(lensdir, 'cframe-{}0-{:08d}.fits'.format(channel, 0)))
    ff_test = desispec.io.read_frame(os.path.join(lensdir, 'cframe-{}0-{:08d}.fits'.format(channel, 1)))
    plt.plot(ff.wave, ff.flux[0, :] * np.sqrt(ff.ivar[0, :]))
    plt.plot(ff_test.wave, ff_test.flux[0, :] * np.sqrt(ff_test.ivar[0, :]))
plt.xlim(6000, 7000)
plt.ylim(0, 25)
plt.show()

